# Welcome to PYNQ

## Getting Started

To get started using PYNQ, try running the example notebooks in the folders described below. 

* **getting_started**: includes an introduction to using Jupyter notebook with PYNQ, the Python environment, and how to use some basic features of the curernt platform. 

* **common**: contains example notebooks on how to download an overlay, how to set the Zynq clocks, how to execute Linux shell commands, and how to use USB devices.

If other overlays or packages are installed, other folders with example notebooks may also be available in this directory.  


## Documentation

Please see the latest <a href="http://pynq.readthedocs.io">PYNQ Documentation on readthedocs</a>.  


## Support

For questions or support, go to the forum on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## Project webpage

You can find details on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## GitHub

The PYNQ Repository is hosted on github: <a href="https://github.com/Xilinx/PYNQ">PYNQ GitHub Repository</a>.

In [1]:
import findspark
findspark.init()


In [2]:

import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
import os

In [5]:
os.getcwd()

'/home/xilinx/jupyter_notebooks'

In [145]:
from pynq import Overlay, Clocks

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')

CPU:   650.000000MHz
FCLK0: 76.923077MHz


In [125]:
# Use bitstream for PL 
overlay = Overlay(os.getcwd() + "/nn3/parkingPynqOverlay/dpu.bit")

In [75]:
overlay?

In [126]:
dpu_ip = overlay.dpu_ip
dpu_ip?

In [127]:
#  Shows the Registers we need to access - Can use access via names or direct memory  (via names is easily)
dpu_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  input1_V_1 = Register(input1_V=0),
  input1_V_2 = Register(input1_V=0),
  layer11_out_0_V = Register(layer11_out_0_V=0),
  layer11_out_0_V_ctrl = Register(layer11_out_0_V_ap_vld=0, RESERVED=0),
  const_size_in_1 = Register(const_size_in_1=0, RESERVED=0),
  const_size_in_1_ctrl = Register(const_size_in_1_ap_vld=0, RESERVED=0),
  const_size_out_1 = Register(const_size_out_1=0, RESERVED=0),
  const_size_out_1_ctrl = Register(const_size_out_1_ap_vld=0, RESERVED=0)
}

In [95]:
help(dpu_ip.register_map.CTRL)

Help on RegisterCTRL in module pynq.registers object:

class RegisterCTRL(Register)
 |  Control signals
 |  
 |  Method resolution order:
 |      RegisterCTRL
 |      Register
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  AP_START
 |      Control signals
 |  
 |  AP_DONE
 |      Control signals
 |  
 |  AP_IDLE
 |      Control signals
 |  
 |  AP_READY
 |      Control signals
 |  
 |  RESERVED_1
 |      Control signals
 |  
 |  AUTO_RESTART
 |      Control signals
 |  
 |  RESERVED_2
 |      Control signals
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  _fields = {'AP_DONE': {'access': 'read-only', 'bit_offset': 1, 'bit_wi...
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Register:
 |  
 |  __getitem__(self, index)
 |      Get the register value.
 |      
 |      This method accepts both integer index, or slice as i

In [135]:
# NN3

# First signal to set high with axi lite IP. Ensures AP_START Signal does not go low after one cycle
dpu_ip.register_map.CTRL.AUTO_RESTART = 1
# Computations occur while high
dpu_ip.register_map.CTRL.AP_START = 1

In [136]:
# Check if signals were set and read only registers have values
dpu_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  input1_V_1 = Register(input1_V=0),
  input1_V_2 = Register(input1_V=0),
  layer11_out_0_V = Register(layer11_out_0_V=0),
  layer11_out_0_V_ctrl = Register(layer11_out_0_V_ap_vld=0, RESERVED=0),
  const_size_in_1 = Register(const_size_in_1=2, RESERVED=0),
  const_size_in_1_ctrl = Register(const_size_in_1_ap_vld=0, RESERVED=0),
  const_size_out_1 = Register(const_size_out_1=1, RESERVED=0),
  const_size_out_1_ctrl = Register(const_size_out_1_ap_vld=0, RESERVED=0)
}

In [140]:
# NN3

## Manually insert timestamp and carpark ID
dpu_ip.register_map.input1_V_1 = 1587568860 
dpu_ip.register_map.input1_V_2 = 1

## Output
output = int(dpu_ip.register_map.layer11_out_0_V)
print(output)

## Testing 2nd input
dpu_ip.register_map.input1_V_1 = 1586032860
dpu_ip.register_map.input1_V_2 = 1

## Output
output = int(dpu_ip.register_map.layer11_out_0_V)
print(output)

## Testing 3rd input
dpu_ip.register_map.input1_V_1 = 1583909160
dpu_ip.register_map.input1_V_2 = 1

## Output
output = int(dpu_ip.register_map.layer11_out_0_V)
print(output)

105
105
105


In [147]:
import time
listOut = []

In [161]:
startTime = time.time()
records = 1000

for i in range(records):
    dpu_ip.register_map.input1_V_1 = 1583909160

    dpu_ip.register_map.input1_V_2 = 1  
    
    ## Output
#     listOut.append(int(dpu_ip.register_map.layer11_out_0_V))
    
endTime = time.time()

In [162]:
print("Time for {0} records in ms: {1}".format(records, endTime - startTime))

Time for 1000 records in ms: 0.40923237800598145


In [160]:
# for i in range(records):
#     print(listOut[i])

In [105]:
# NN2 - already tested all predictions quantise to 0 
# as we didnt allocate any bits for floating pt for weights in hls4m it made all weights 0 

In [126]:
from decimal import Decimal
# neural_network - 64 bit i/0 (32bit for integer, 32 for dec point),likewise for weights and activations
# I/O is little endian


# Configure the ip block to be continously accept inputs (no stop-start)
dpu_ip.register_map.CTRL.AUTO_RESTART = 1
dpu_ip.register_map.CTRL.AP_START = 1 

# Manually insert timestamp and carpark ID

# timestamp
dpu_ip.register_map.input1_V_2 = 1587568890
dpu_ip.register_map.input1_V_1 = 0

# carpark ID
dpu_ip.register_map.input1_V_4=1
dpu_ip.register_map.input1_V_3=0




# Prediction Output (Should print 0 as we have quantised - 32 bit for integer 0 for decimal hence all weights =0 )
# floating part 

# integer part
from  
integ = dpu_ip.register_map.layer10_out_0_V_2
dec = dpu_ip.register_map.layer10_out_0_V_1
# print("Prediction: "+str(integ)+"."+str(dec))

# Test 2nd Input
dpu_ip.register_map.input1_V_2 = 1587373860
dpu_ip.register_map.input1_V_1 = 0

dpu_ip.register_map.input1_V_4=1
dpu_ip.register_map.input1_V_3=0
# dpu_ip.register_map.layer10_out_0_V_1
# dpu_ip.register_map.layer10_out_0_V_2


print(int(dec))

0x9224d8bb


Register(layer10_out_0_V=2451888315)

In [122]:
dpu_ip.register_map.layer10_out_0_V_1.width

32

In [12]:
dpu_ip.register_map.input1_V_1.width

32